In [3]:
###This below code can be used to adjust the width of the containers for display purposes.
#from IPython.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

################################################################################
# Matthew Spitulnik ############################################################
# Natural Language Processing ##################################################
# Comparing Corpora with Corpus Statistics #####################################
# Project Summary: This project is a continuation of a previous project, Comparing Corpora with Corpus Statistics (CCCS). For CCCS, the song lyrics from the 100 top billboard songs from each year between 1980 and 2019 were collected and compiled into one data frame. They were then split into two groups, the years before the year 2000, and the years after/including the year 2000. Some corpus statistics were then performed on the words in each group of lyrics, which were then analyzed to see if anything or any patterns stood out that seperated the song lyrics before the year 2000 and the song lyrics including/after the year 2000. For this assessment, the same collection of song lyrics will be utilized to compare the lyrics before and after the year 2000, but different parts of speech analysis will be applied to the text to identify and pull out lyrics containting adjective/adverb part of speech phrases, determine the most common adjectives/adverbs/nouns/verbs used in both time frames, and then perform sentiment analysis on the data. Unlike in CCCS, the analysis of this data will be performed and reported on within in this code file instead of a seperate written report.
################################################################################



################################################################################
### Install and load required packages #########################################
################################################################################

In [ ]:
###Install the required libraries.
#%pip install pandas
#%pip install re
#%pip install requests
#%pip install html5lib
#%pip install nltk
#%pip install contractions
#%pip install lxml
#%pip install bs4

In [2]:
#Import the required libraries
import pandas as pd
import re
import requests
import html5lib
import nltk
#import sys
#!{sys.executable} -m pip install contractions
import contractions
from nltk import FreqDist
from lxml import html
from urllib.request import urlopen
from urllib import request
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.collocations import *
from nltk import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('stopwords', quiet=True)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\spitum1\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

################################################################################
### Import the data sets that will be used throughout the code #################
################################################################################

In [ ]:
###This creates a "default_directory" variable, where the directory path to the 
# data files folder containing all of the required data sets is saved so that it 
# does not need to be constantly re-entered. Remember to use forward slashes 
# instead of back slashes in the directory path. For example, if the datafiles 
# folder is saved in "C:\home\project\datafiles", then "C:/home/project" 
# would be inserted between the quotes below.
default_directory = "<DEFAULT DIRECTORY PATH HERE>"

In [4]:
#load the data frames that were used in CCCS
songArtDF = pd.read_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtDF.csv')
songArtDFOrg = pd.read_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtDFOrg.csv')
songArtLyrics =pd.read_csv(f'{default_directory}/Comparing Corpora With Corpus Statistics/datafiles/songArtLyrics.csv')

################################################################################
### Perform Data Cleanup #######################################################
################################################################################

In [5]:
#make a copy of the song lyrics DF that I will use to turn everything into a sentence
songArtsent=songArtDFOrg.copy(deep=True)
songArtsent=songArtsent[songArtsent['Lyrics'] != '[]']

In the final data frame containing the song lyrics for each song, a categorical/factor variable was added that simply identified if each song was from before or after/in the year 2000.

In [7]:
#make sure we have the correct number of songs with lyrics
print(songArtsent['Before2000'].value_counts()['Yes']) #1951 songs before 2000
print(songArtsent['Before2000'].value_counts()['No']) #1849 songs after/including 2000

1951
1849


In theory, since 40 years of the 100 top billboard songs were looked at, there should of been 40 x 100 songs with lyrics, for a total of 4000. 2000 songs before the year 2000, 2000 songs after/including the year 2000. Some different scripting techniques needed to be utilized to collect as much of the data as possible, and above shows that the lyrics could only be collected for 1951 of the songs before the year 2000, and the lyrics could be collected for 1849 of the songs after the year 2000.

In [9]:
for i in range(0,3):
    print(songArtDF.loc[i,'Lyrics'])

['\nColour me your colour, baby', 'Colour me your car', 'Colour me your colour, darling', 'I know who you are', 'Come up off your colour chart', "I know where you're comin' from", 'Call me (call me) on the line', 'Call me, call me any, anytime', 'Call me (call me) my love', 'You can call me any day or night', 'Call me']
["\nWe don't need no education ", "We don't need no thought control ", 'No dark sarcasm in the classroom ', 'Teachers leave the kids alone ', 'Hey teacher leave us kids alone ', "All in all it's just another brick in the wall ", "All in all you're just another brick in the wall "]
['\nCome take my hand', '\nYou should know me', "\nI've always been in your mind", "\nYou know that I'll be kind", "\nI'll be guiding you", '\n']


The text for the song lyric data that was imported was still pretty raw and had not gone through any kind of cleaning process yet. The big issue in terms of clean up was going to be to format the data in a way that the song lyrics could be broken up into multiple sentences instead of just one long run on sentence. In the data above, the song lyrics from 3 songs are displayed. The lyrics for each song starts with an open bracket [ and concludes with a closed bracket ]. In between [ and ] are the song lyrics. Each line of the lyrics are surrounded by either single quotes '' or double quotes "", followed by a comma, unless it is the last line of a song, in which case the ' or " is followed by the ]. The nltk.sent_tokenize function, which is usd to break strings of text into individual sentences, looks for punctuation that is generally at the end of sentences, like periods . exclamation points ! and question marks ? to accomplish this. When using nltk.sent_tokenize, wherever these types of punctuation exist is where a sentence is viewed as ending.

Since this punctuation did not exist in the song lyric text, something would need to be done to get one of these line-ending-types of punctuation inserted where each lyrical line ended. The first thought was to replace some of the different line break markers like '\n' or '\r' with a period. It was quickly discovered however that in many of the song lyric texts, these line break markers only appeared at the beginning or the end of the entire song lyric, not in between each line. The next obersvation was that each line in a lyric appeared to start with a capital letter, so if regex could be used to find every capital letter in the song lyrics, a period could be inserted before it, and then nltk.sent_tokenize could identify where the sentence before it ended. Unfortunately after further investgation it was determined that this capital letter pattern was not consistent, with other words throughout song lyrics containing capital letters even though they weren't the first word. Any sentence that had some form of 'I' throughout it would also throw off this pattern.

It seemed like the best course of action was to use the observation stated above, that each line of a lyric was surrounded by '' or "" followed by a comma. This pattern appeared to be consistent in every song lyric that was studied. There also did not appear to be any stray instances of ', or ", appearing in a location that was not at the end of a line/sentence. Even if an abbreviated or slang word like talkin' appeared at the end of a line, it would still be followed by another single or double quote, then a comma, appearing as so: talkin'', or talkin'",. The script that is created to place the period would still just replace the ', or ", with a period. With that figured out, the following steps were taken to clean up the data:

1) Remove line break text like \n and \r or excessive backslashes like \\.
2) Replace contractions like can't and don't with cannot and do not so that punctuation can later be easily removed without losing the meaning of words.
3) Replace all instances of ', with a period . to signify the end of a line.
4) Replace all instances of ", with a period . to signify the end of a line.
5) Replace all instances of '] and "] at the end of a song's lyrics with a period.
6) Seperate the lyrics into two lists, the lyrics for songs before the year 2000 and the lyrics for songs after/including the year 2000.
7) Split the songs lyrics for before 2000 into individual sentences, which was compiled into a list of all the sentences.
8) Split the songs lyrics for after 2000 into individual sentences, which was compiled into a list of all the sentences.

Additional data manipulation that was performed will be explained further down in the code.

In [12]:
#remove line breaks again like I did in CCCS
for i in songArtsent.index:
    songArtsent.loc[i,'Lyrics']=songArtsent.loc[i,'Lyrics'].replace('\\n','')
    songArtsent.loc[i,'Lyrics']=songArtsent.loc[i,'Lyrics'].replace('\\r','')
    songArtsent.loc[i,'Lyrics']=songArtsent.loc[i,'Lyrics'].replace('\\','')

In [13]:
#remove contractions again so other punctation can be easily removed later
for i in songArtsent.index:
    songArtsent.loc[i,'Lyrics']=contractions.fix(songArtsent.loc[i,'Lyrics'])

In [14]:
for i in range(0,10):
    print(songArtsent.loc[i,'Lyrics'])

['Colour me your colour, baby', 'Colour me your car', 'Colour me your colour, darling', 'I know who you are', 'Come up off your colour chart', "I know where you are comin' from", 'Call me (call me) on the line', 'Call me, call me any, anytime', 'Call me (call me) my love', 'You can call me any day or night', 'Call me']
["We do not need no education ", "We do not need no thought control ", 'No dark sarcasm in the classroom ', 'Teachers leave the kids alone ', 'Hey teacher leave us kids alone ', "All in all it is just another brick in the wall ", "All in all you are just another brick in the wall "]
['Come take my hand', 'You should know me', "I have always been in your mind", "You know that I will be kind", "I will be guiding you", '']
['[1st Verse]', 'Girl, Close Your Eyes', 'Let That Rhythm Get Into You', "Do not Try To Fight It", "There Are not Nothing That You Can Do", 'Relax Your Mind', 'Lay Back And Groove With Mine', 'You Got To Feel That Heat', 'And We Can Ride The Boogie', 'Sha

In [15]:
####next few cells are test regex searches

for i in range(8,9):
    apostrapheReplaceTest=re.findall('[\',]',songArtsent.loc[i,'Lyrics'])
    
apostrapheReplaceTest

[',',
 "'",
 "'",
 "'",
 ',',
 "'",
 "'",
 ',',
 "'",
 "'",
 ',',
 "'",
 ',',
 ',',
 "'",
 "'",
 ',',
 "'",
 "'",
 "'",
 ',',
 ',',
 ',',
 "'",
 "'",
 ',',
 ',',
 "'",
 "'",
 ',',
 "'",
 "'",
 ',',
 "'",
 "'",
 ',',
 ',',
 "'",
 "'",
 "'",
 ',',
 ',',
 ',',
 ',',
 ',',
 "'",
 "'",
 ',',
 ',',
 ',',
 ',',
 ',',
 ',',
 "'",
 "'",
 ',',
 "'",
 "'",
 ',',
 "'",
 "'",
 ',',
 "'",
 "'",
 ',',
 "'",
 "'",
 ',',
 "'",
 "'",
 ',',
 ',',
 ',',
 "'",
 "'",
 ',',
 "'",
 ',',
 ',',
 ',',
 "'",
 ',',
 ',',
 "'",
 "'",
 ',',
 ',',
 "'",
 "'",
 ',',
 "'",
 "'",
 ',',
 "'",
 "'",
 ',',
 ',',
 "'",
 ',',
 ',',
 ',',
 ',',
 ',',
 ',',
 "'",
 "'",
 ',',
 "'",
 "'",
 "'",
 ',',
 ',']

In [16]:
for i in range(8,9):
    apostrapheReplaceTest=re.findall('\'[\',]',songArtsent.loc[i,'Lyrics'])
    
apostrapheReplaceTest

["',",
 "',",
 "',",
 "',",
 "''",
 "',",
 "',",
 "',",
 "',",
 "''",
 "',",
 "''",
 "',",
 "',",
 "',",
 "',",
 "',",
 "',",
 "''",
 "',",
 "',",
 "',",
 "''"]

In [17]:
for i in range(8,9):
    apostrapheReplaceTest=re.findall('\'[^,]',songArtsent.loc[i,'Lyrics'])
    
apostrapheReplaceTest

['\'"',
 "'s",
 "'M",
 '\'"',
 "' ",
 "'Y",
 "'s",
 "' ",
 "'b",
 "''",
 '\'"',
 "'S",
 '\'"',
 "'Y",
 "'s",
 "''",
 "'A",
 "''",
 "'H",
 "'A",
 '\'"',
 "'I",
 "'Y",
 "'N",
 "''",
 '\'"',
 "'A",
 "' ",
 '\'"',
 "''",
 "'s",
 "' ",
 "'b"]

In [18]:
######now do some sub regex testing
re.sub('\'[\",]','.',songArtsent.loc[8,'Lyrics'])

'["What is the matter with the clothes I am wearing?", .Cannot you tell that your tie\'s too wide?". \'Maybe I should buy some old tab collars?. .Welcome back to the age of jive.. "Where have you been hidin\' out lately, honey?", \'You cannot dress trashy till you spend a lot of money.". "Everybody\'s talkin\' \'bout the new sound", "Funny, but it is still rock and roll to me", \'. "What is the matter with the car I am driving?", .Cannot you tell that it is out of style?". \'Should I get a set of white wall tires?. .Are you going to cruise the miracle mile?. "Nowadays you cannot be too sentimental", \'Your best bet\'s a true baby blue Continental.". "Hot funk, cool punk, even if it is old junk", "It is still rock and roll to me", \'. "Oh, it does not matter what they say in the papers", "Because it is always been the same old scene.", "There is a new band in town", "But you cannot get the sound from a story in a magazine...", \'Aimed at your average teen. \'. \'How about a pair of pink

In [19]:
re.sub('\",','.',songArtsent.loc[8,'Lyrics'])

'["What is the matter with the clothes I am wearing?. \'"Cannot you tell that your tie\'s too wide?"\', \'Maybe I should buy some old tab collars?\', \'"Welcome back to the age of jive.\', "Where have you been hidin\' out lately, honey?. \'You cannot dress trashy till you spend a lot of money."\', "Everybody\'s talkin\' \'bout the new sound. "Funny, but it is still rock and roll to me. \'\', "What is the matter with the car I am driving?. \'"Cannot you tell that it is out of style?"\', \'Should I get a set of white wall tires?\', \'"Are you going to cruise the miracle mile?\', "Nowadays you cannot be too sentimental. \'Your best bet\'s a true baby blue Continental."\', "Hot funk, cool punk, even if it is old junk. "It is still rock and roll to me. \'\', "Oh, it does not matter what they say in the papers. "Because it is always been the same old scene.. "There is a new band in town. "But you cannot get the sound from a story in a magazine.... \'Aimed at your average teen\', \'\', \'How 

In [20]:
re.sub('\'[\',]','.',songArtsent.loc[0,'Lyrics'])

'[\'Colour me your colour, baby. \'Colour me your car. \'Colour me your colour, darling. \'I know who you are. \'Come up off your colour chart. "I know where you are comin\' from", \'Call me (call me) on the line. \'Call me, call me any, anytime. \'Call me (call me) my love. \'You can call me any day or night. \'Call me\']'

In [21]:
re.sub('\',','.',songArtsent.loc[0,'Lyrics'])

'[\'Colour me your colour, baby. \'Colour me your car. \'Colour me your colour, darling. \'I know who you are. \'Come up off your colour chart. "I know where you are comin\' from", \'Call me (call me) on the line. \'Call me, call me any, anytime. \'Call me (call me) my love. \'You can call me any day or night. \'Call me\']'

In [22]:
####now run the sub regex statement to replace '\',' with a '.'
for i in songArtsent.index:
    songArtsent.loc[i,'Lyrics']=re.sub('\',','.',songArtsent.loc[i,'Lyrics'])

In [23]:
songArtsent.loc[0,'Lyrics']

'[\'Colour me your colour, baby. \'Colour me your car. \'Colour me your colour, darling. \'I know who you are. \'Come up off your colour chart. "I know where you are comin\' from", \'Call me (call me) on the line. \'Call me, call me any, anytime. \'Call me (call me) my love. \'You can call me any day or night. \'Call me\']'

In [24]:
songArtsent.loc[8,'Lyrics']

'["What is the matter with the clothes I am wearing?", \'"Cannot you tell that your tie\'s too wide?". \'Maybe I should buy some old tab collars?. \'"Welcome back to the age of jive.. "Where have you been hidin\' out lately, honey?", \'You cannot dress trashy till you spend a lot of money.". "Everybody\'s talkin\' \'bout the new sound", "Funny, but it is still rock and roll to me", \'. "What is the matter with the car I am driving?", \'"Cannot you tell that it is out of style?". \'Should I get a set of white wall tires?. \'"Are you going to cruise the miracle mile?. "Nowadays you cannot be too sentimental", \'Your best bet\'s a true baby blue Continental.". "Hot funk, cool punk, even if it is old junk", "It is still rock and roll to me", \'. "Oh, it does not matter what they say in the papers", "Because it is always been the same old scene.", "There is a new band in town", "But you cannot get the sound from a story in a magazine...", \'Aimed at your average teen. \'. \'How about a pair

In [25]:
####now run the sub regex statement to replace '\",' with a '.'
for i in songArtsent.index:
    songArtsent.loc[i,'Lyrics']=re.sub('\",','.',songArtsent.loc[i,'Lyrics'])

In [26]:
songArtsent.loc[0,'Lyrics']

'[\'Colour me your colour, baby. \'Colour me your car. \'Colour me your colour, darling. \'I know who you are. \'Come up off your colour chart. "I know where you are comin\' from. \'Call me (call me) on the line. \'Call me, call me any, anytime. \'Call me (call me) my love. \'You can call me any day or night. \'Call me\']'

In [27]:
songArtsent.loc[8,'Lyrics']

'["What is the matter with the clothes I am wearing?. \'"Cannot you tell that your tie\'s too wide?". \'Maybe I should buy some old tab collars?. \'"Welcome back to the age of jive.. "Where have you been hidin\' out lately, honey?. \'You cannot dress trashy till you spend a lot of money.". "Everybody\'s talkin\' \'bout the new sound. "Funny, but it is still rock and roll to me. \'. "What is the matter with the car I am driving?. \'"Cannot you tell that it is out of style?". \'Should I get a set of white wall tires?. \'"Are you going to cruise the miracle mile?. "Nowadays you cannot be too sentimental. \'Your best bet\'s a true baby blue Continental.". "Hot funk, cool punk, even if it is old junk. "It is still rock and roll to me. \'. "Oh, it does not matter what they say in the papers. "Because it is always been the same old scene.. "There is a new band in town. "But you cannot get the sound from a story in a magazine.... \'Aimed at your average teen. \'. \'How about a pair of pink sid

In [28]:
#now test removing the ']' at the end of each song's lyrics
re.sub('\']','.',songArtsent.loc[0,'Lyrics'])

'[\'Colour me your colour, baby. \'Colour me your car. \'Colour me your colour, darling. \'I know who you are. \'Come up off your colour chart. "I know where you are comin\' from. \'Call me (call me) on the line. \'Call me, call me any, anytime. \'Call me (call me) my love. \'You can call me any day or night. \'Call me.'

In [29]:
re.sub('\"]','.',songArtsent.loc[8,'Lyrics'])

'["What is the matter with the clothes I am wearing?. \'"Cannot you tell that your tie\'s too wide?". \'Maybe I should buy some old tab collars?. \'"Welcome back to the age of jive.. "Where have you been hidin\' out lately, honey?. \'You cannot dress trashy till you spend a lot of money.". "Everybody\'s talkin\' \'bout the new sound. "Funny, but it is still rock and roll to me. \'. "What is the matter with the car I am driving?. \'"Cannot you tell that it is out of style?". \'Should I get a set of white wall tires?. \'"Are you going to cruise the miracle mile?. "Nowadays you cannot be too sentimental. \'Your best bet\'s a true baby blue Continental.". "Hot funk, cool punk, even if it is old junk. "It is still rock and roll to me. \'. "Oh, it does not matter what they say in the papers. "Because it is always been the same old scene.. "There is a new band in town. "But you cannot get the sound from a story in a magazine.... \'Aimed at your average teen. \'. \'How about a pair of pink sid

In [30]:
####now  replace '']' and '"]' with a '.'
for i in songArtsent.index:
    songArtsent.loc[i,'Lyrics']=re.sub('\']','.',songArtsent.loc[i,'Lyrics'])

for i in songArtsent.index:
    songArtsent.loc[i,'Lyrics']=re.sub('\"]','.',songArtsent.loc[i,'Lyrics'])

In [31]:
songArtsent.to_csv(f'{default_directory}/Sentiment and Exploratory Analysis/datafiles/songArtsent.csv',index=False)

In [32]:
#combine all the lyrics into two lists of lyrics, before 2000 and after 2000.
before2000lyr=[]
after2000lyr=[]
for i in songArtsent.index:
    if songArtsent.loc[i,'Before2000']=='Yes':
        before2000lyr.append(songArtsent.loc[i,'Lyrics'])
    else:
        after2000lyr.append(songArtsent.loc[i,'Lyrics'])

In [33]:
print(before2000lyr[:3])

['[\'Colour me your colour, baby. \'Colour me your car. \'Colour me your colour, darling. \'I know who you are. \'Come up off your colour chart. "I know where you are comin\' from. \'Call me (call me) on the line. \'Call me, call me any, anytime. \'Call me (call me) my love. \'You can call me any day or night. \'Call me.', '["We do not need no education . "We do not need no thought control . \'No dark sarcasm in the classroom . \'Teachers leave the kids alone . \'Hey teacher leave us kids alone . "All in all it is just another brick in the wall . "All in all you are just another brick in the wall .', '[\'Come take my hand. \'You should know me. "I have always been in your mind. "You know that I will be kind. "I will be guiding you. \'.']


In [34]:
#split the Before Lyrics into sentences
BLSplit = nltk.sent_tokenize(str(before2000lyr))

In [35]:
print(BLSplit[0:12])

["['[\\'Colour me your colour, baby.", "\\'Colour me your car.", "\\'Colour me your colour, darling.", "\\'I know who you are.", "\\'Come up off your colour chart.", '"I know where you are comin\\\' from.', "\\'Call me (call me) on the line.", "\\'Call me, call me any, anytime.", "\\'Call me (call me) my love.", "\\'You can call me any day or night.", "\\'Call me.", '\', \'["We do not need no education .']


In [36]:
print(BLSplit[10:11])

["\\'Call me."]


In [37]:
print(BLSplit[11:12])

['\', \'["We do not need no education .']


In [38]:
#split the After Lyrics into sentences
ALSplit = nltk.sent_tokenize(str(after2000lyr))

In [39]:
print(ALSplit[:10])

["['[\\'I can feel the magic floating in the air.", "\\'Being with you gets me that way.", '"I watch the sunlight dance across your face and I have.', "\\'Never been this swept away.", "\\'.", 'my thoughts just seem to settle on the breeze.', '"When I am lying wrapped up in your arms.', "\\'The whole world just fades away.", "\\'The only thing I hear.", "\\'Is the beating of your heart."]


In [40]:
print(ALSplit[0:1])

["['[\\'I can feel the magic floating in the air."]


In [41]:
print(ALSplit[1])

\'Being with you gets me that way.


In [42]:
len(BLSplit)

92224

In [43]:
BLSplit[92223]

"\\'.']"

In [44]:
BLSplit[92222]

"\\'God must have spent a little more time on you."

In [45]:
BLSplit[len(BLSplit)-1:len(BLSplit)]

["\\'.']"]

Now that the before 2000 and after 2000 lyrics had been broken up into individual sentences, punctuation was no longer needed within the text. Because of how the punctuation lined up after some of the above clean up was performed, some entries in the list of lyric sentences only contained punctuation, so these entries were blank after the punctuation removal and were subsequently removed from the lists of before 2000 sentences and after 2000 sentences. What remained was 77891 sentences from lyrics before the year 2000 and 111133 sentences from lyrics after the year 2000. This was interesting to see, since lyrics for over 100 more songs before the year 2000 were found than lyrics for songs after the year 2000, and yet the lyrics for songs after the year 2000 resulted in over 30000 more sentences. This translated to about 40 lines per song before 2000 and about 60 lines per song after the year 2000.

In [47]:
#now that the lyrics have been split up into sentects, remove all of the punctuation
#first the before lyrics
for i in range(0,len(BLSplit)):
    BLSplit[i]=re.sub(r'[^\w\s]','',BLSplit[i])

In [48]:
BLSplit[92223]

''

In [49]:
len(BLSplit)

92224

In [50]:
for i in range(92210,len(BLSplit)):
    print(BLSplit[i])

God must of spent A little more time On you

Never thought that love could feel like this
Then you change my world with just one kiss
How can it be that right here with me 
There Is an angel is just a miracle

Your love is like a river peaceful and deep
Your soul is like a secret That I could never keep
When I look into your eyes I know that it is true
God must of spent A little more time On you

God must have spent a little more time on you



In [51]:
#Remove before lyrics list entries that are blank
for i in range(92210,len(BLSplit)):
    if BLSplit[i]!='':
        print('List entry: ', i, ' || ', BLSplit[i])

List entry:  92210  ||  God must of spent A little more time On you
List entry:  92212  ||  Never thought that love could feel like this
List entry:  92213  ||  Then you change my world with just one kiss
List entry:  92214  ||  How can it be that right here with me 
List entry:  92215  ||  There Is an angel is just a miracle
List entry:  92217  ||  Your love is like a river peaceful and deep
List entry:  92218  ||  Your soul is like a secret That I could never keep
List entry:  92219  ||  When I look into your eyes I know that it is true
List entry:  92220  ||  God must of spent A little more time On you
List entry:  92222  ||  God must have spent a little more time on you


In [52]:
BLSplitR=[]
for i in range(0,len(BLSplit)):
    if BLSplit[i]!='':
        BLSplitR.append(BLSplit[i])

In [53]:
len(BLSplitR)

77891

In [54]:
for i in range(77770,len(BLSplitR)):
    if BLSplitR[i]!='':
        print('List entry: ', i, ' || ', BLSplitR[i])

List entry:  77770  ||  I have been the star of many plays
List entry:  77771  ||  I have seen better days
List entry:  77772  ||  And the bottom drops out
List entry:  77773  ||  I have seen better days
List entry:  77774  ||  I have been the star of many plays
List entry:  77775  ||  I have seen better days
List entry:  77776  ||  I have seen better days
List entry:  77777  ||  I know that I have seen better days
List entry:  77778  ||    And the bottom drops out
List entry:  77779  ||  I have been the star of so many plays
List entry:  77780  ||    And the bottom drops out
List entry:  77781  ||  I walked on the edge without a hold whole way
List entry:  77782  ||    And the botton drops out
List entry:  77783  ||  Cuz I know I know that I have seen better better days
List entry:  77784  ||    And the bottom drops out
List entry:  77785  ||   Dana HuntKent M Robbins
List entry:  77786  ||  I never saw the end in sight fools are kind of blind  Thought everything was going alright but

In [55]:
#Remove punctuation in after lyrics
for i in range(0,len(ALSplit)):
    ALSplit[i]=re.sub(r'[^\w\s]','',ALSplit[i])

In [56]:
len(ALSplit)

129634

In [57]:
#Remove punctuation in after lyrics
for i in range(129600,len(ALSplit)):
    print(ALSplit[i])

Scootie  Ariana Grande
I want to kiss you Yeah do not want to miss you Yeah
But I cannot be with you because I got issues
Yeah on the surface seem like it is easy
Careful with words but it is still hard to read me
Stress high when the trust low Mmm
Bad vibes where did the fun go
Oh
Try to open up and love more Love more
Try to open up and love more

Scootie Ariana Grande  Both
If you were my boyfriend
And you were my girlfriend
I probably would not see nobody else
But I cannot guarantee that by myself

Ariana Grande
You are not my boyfriend Boyfriend you are not my boyfriend
And I are not your girlfriend Girlfriend I are not your girlfriend
But you do not want me to see nobody else Nobody
And I do not want you to see nobody
But you are not my boyfriend Boyfriend you know you are not my boyfriend
And I are not your girlfriend Girlfriend yeah mmm
But you do not want me to touch nobody else Nobody
Baby we are not got to tell nobody Oh yeah
You are not my boyfriend Boyfriend
And I are not 

In [58]:
ALSplitR=[]
for i in range(0,len(ALSplit)):
    if ALSplit[i]!='':
        ALSplitR.append(ALSplit[i])

In [59]:
len(ALSplitR)

111133

################################################################################
### Perform Data Analysis ######################################################
################################################################################

In [60]:
#figure out how many lines per song, on average, for before and after 2000
#number of songs with lyrics before 2000: 1951
#number of songs with lyrics after 2000: 1849

print('Average number of lines per song before the year 2000:', len(BLSplitR) / 1951)
print('Average number of lines per song after the year 2000:', len(ALSplitR) / 1849)

Average number of lines per song before the year 2000: 39.92362890825218
Average number of lines per song after the year 2000: 60.10438074634938


In [61]:
for i in range(111100,len(ALSplitR)):
    print(ALSplitR[i])

Baby we are not got to tell nobody
Scootie  Ariana Grande
I want to kiss you Yeah do not want to miss you Yeah
But I cannot be with you because I got issues
Yeah on the surface seem like it is easy
Careful with words but it is still hard to read me
Stress high when the trust low Mmm
Bad vibes where did the fun go
Oh
Try to open up and love more Love more
Try to open up and love more
Scootie Ariana Grande  Both
If you were my boyfriend
And you were my girlfriend
I probably would not see nobody else
But I cannot guarantee that by myself
Ariana Grande
You are not my boyfriend Boyfriend you are not my boyfriend
And I are not your girlfriend Girlfriend I are not your girlfriend
But you do not want me to see nobody else Nobody
And I do not want you to see nobody
But you are not my boyfriend Boyfriend you know you are not my boyfriend
And I are not your girlfriend Girlfriend yeah mmm
But you do not want me to touch nobody else Nobody
Baby we are not got to tell nobody Oh yeah
You are not my b

Now that the lyrics were broken into individual sentences and cleaned up, steps needed to be taken to identify the words that make up different parts of speech within each sentence. To do this, each sentence in the before 2000 list and the after 2000 list needed to be tokenized once again to break them down into individual words. This resulted in a list of lists, with the overall list containing the lyrics of each individual song, and then within each song list entry, each lyric line was was another list with each word as an entry. For example, one song now appeared like this: 

[['Colour', 'me', 'your', 'colour', 'baby'], ['Colour', 'me', 'your', 'car'], ['Colour', 'me', 'your', 'colour', 'darling'], ['I', 'know', 'who', 'you', 'are'], ['Come', 'up', 'off', 'your', 'colour', 'chart'], ['I', 'know', 'where', 'you', 'are', 'comin', 'from'], ['Call', 'me', 'call', 'me', 'on', 'the', 'line'], ['Call', 'me', 'call', 'me', 'any', 'anytime'], ['Call', 'me', 'call', 'me', 'my', 'love'], ['You', 'can', 'call', 'me', 'any', 'day', 'or', 'night']]

Next, all words, except for "I", were changed to be lowercase, so a word at the beginning of a sentence that was capitalized would be counted just the same as if it was in the middle of a sentence. Next, nltk.pos_tag was applied to both lists of lyric words, which took every single word, identified it's part of speech, then created tuples for each word containing the word and the symbol for its part of speech. The information now appeared like this: 

[[('colour', 'VB'), ('me', 'PRP'), ('your', 'PRP$'), ('colour', 'NN'), ('baby', 'NN')], [('colour', 'VB'), ('me', 'PRP'), ('your', 'PRP$'), ('car', 'NN')]]

In [63]:
# Apply the word tokenizer to each sentence in the Before Lyrics
BLtoken = [nltk.word_tokenize(sent) for sent in BLSplitR]
print(BLtoken[:10])
#the output is a list of strings that contains the sentences
print(type(BLtoken))
len(BLtoken)

[['Colour', 'me', 'your', 'colour', 'baby'], ['Colour', 'me', 'your', 'car'], ['Colour', 'me', 'your', 'colour', 'darling'], ['I', 'know', 'who', 'you', 'are'], ['Come', 'up', 'off', 'your', 'colour', 'chart'], ['I', 'know', 'where', 'you', 'are', 'comin', 'from'], ['Call', 'me', 'call', 'me', 'on', 'the', 'line'], ['Call', 'me', 'call', 'me', 'any', 'anytime'], ['Call', 'me', 'call', 'me', 'my', 'love'], ['You', 'can', 'call', 'me', 'any', 'day', 'or', 'night']]
<class 'list'>


77891

In [64]:
# Apply the word tokenizer to each sentence in after lyrics
ALtoken = [nltk.word_tokenize(sent) for sent in ALSplitR]
print(ALtoken[:10])
#the output is a list of strings that contains the sentences
print(type(ALtoken))
len(ALtoken)

[['I', 'can', 'feel', 'the', 'magic', 'floating', 'in', 'the', 'air'], ['Being', 'with', 'you', 'gets', 'me', 'that', 'way'], ['I', 'watch', 'the', 'sunlight', 'dance', 'across', 'your', 'face', 'and', 'I', 'have'], ['Never', 'been', 'this', 'swept', 'away'], ['my', 'thoughts', 'just', 'seem', 'to', 'settle', 'on', 'the', 'breeze'], ['When', 'I', 'am', 'lying', 'wrapped', 'up', 'in', 'your', 'arms'], ['The', 'whole', 'world', 'just', 'fades', 'away'], ['The', 'only', 'thing', 'I', 'hear'], ['Is', 'the', 'beating', 'of', 'your', 'heart'], ['Because', 'I', 'can', 'feel', 'you', 'breathe']]
<class 'list'>


111133

In [65]:
#now make everything the same case, except for "I"
#before lyrics first
for i in range(0,len(BLtoken)):
    for h in range(0,len(BLtoken[i])):
        if BLtoken[i][h] != 'I':
            BLtoken[i][h]=BLtoken[i][h].lower()

In [66]:
#now make everything the same case, except for "I"
#now after lyrics
for i in range(0,len(ALtoken)):
    for h in range(0,len(ALtoken[i])):
        if ALtoken[i][h] != 'I':
            ALtoken[i][h]=ALtoken[i][h].lower()

In [67]:
print(ALtoken[:10])

[['I', 'can', 'feel', 'the', 'magic', 'floating', 'in', 'the', 'air'], ['being', 'with', 'you', 'gets', 'me', 'that', 'way'], ['I', 'watch', 'the', 'sunlight', 'dance', 'across', 'your', 'face', 'and', 'I', 'have'], ['never', 'been', 'this', 'swept', 'away'], ['my', 'thoughts', 'just', 'seem', 'to', 'settle', 'on', 'the', 'breeze'], ['when', 'I', 'am', 'lying', 'wrapped', 'up', 'in', 'your', 'arms'], ['the', 'whole', 'world', 'just', 'fades', 'away'], ['the', 'only', 'thing', 'I', 'hear'], ['is', 'the', 'beating', 'of', 'your', 'heart'], ['because', 'I', 'can', 'feel', 'you', 'breathe']]


In [68]:
## POS Tagging, to retrieve adjective (JJs) and adverb (RBs) tags

# use the Stanford POS tagger to POS tag tokens of each sentence in the Before Lyrics
# this is the default tagger in nltk
BLtagged = [nltk.pos_tag(tokens) for tokens in BLtoken]

In [69]:
print(BLtagged[:2])

[[('colour', 'VB'), ('me', 'PRP'), ('your', 'PRP$'), ('colour', 'NN'), ('baby', 'NN')], [('colour', 'VB'), ('me', 'PRP'), ('your', 'PRP$'), ('car', 'NN')]]


In [70]:
print(len(BLtagged))
print(len(BLtoken))

77891
77891


In [71]:
# use the Stanford POS tagger to POS tag tokens of each sentence in the After Lyrics
# this is the default tagger in nltk
ALtagged = [nltk.pos_tag(tokens) for tokens in ALtoken]

In [72]:
print(ALtagged[:100])

[[('I', 'PRP'), ('can', 'MD'), ('feel', 'VB'), ('the', 'DT'), ('magic', 'NN'), ('floating', 'NN'), ('in', 'IN'), ('the', 'DT'), ('air', 'NN')], [('being', 'VBG'), ('with', 'IN'), ('you', 'PRP'), ('gets', 'VBZ'), ('me', 'PRP'), ('that', 'IN'), ('way', 'NN')], [('I', 'PRP'), ('watch', 'VBP'), ('the', 'DT'), ('sunlight', 'NN'), ('dance', 'NN'), ('across', 'IN'), ('your', 'PRP$'), ('face', 'NN'), ('and', 'CC'), ('I', 'PRP'), ('have', 'VBP')], [('never', 'RB'), ('been', 'VBN'), ('this', 'DT'), ('swept', 'VBZ'), ('away', 'RB')], [('my', 'PRP$'), ('thoughts', 'NNS'), ('just', 'RB'), ('seem', 'VBP'), ('to', 'TO'), ('settle', 'VB'), ('on', 'IN'), ('the', 'DT'), ('breeze', 'NN')], [('when', 'WRB'), ('I', 'PRP'), ('am', 'VBP'), ('lying', 'VBG'), ('wrapped', 'VBD'), ('up', 'RP'), ('in', 'IN'), ('your', 'PRP$'), ('arms', 'NNS')], [('the', 'DT'), ('whole', 'JJ'), ('world', 'NN'), ('just', 'RB'), ('fades', 'VBZ'), ('away', 'RP')], [('the', 'DT'), ('only', 'JJ'), ('thing', 'NN'), ('I', 'PRP'), ('hear'

Now that the parts of speech of each word had been tagged, information could be pulled out to analyze. First, adjective phrases (a phrase containing an adverb and adjective) were pulled out from both the before 2000 and after 2000 lyric lists. The number of times each adjective phrase appeared in both lists was tallied together using a frequency distribution, which could then be used to compare the two different time periods of lyrics. For both the before 2000 and after 2000 lyrics, the most common adjective phrase was "so much". In fact, for both periods of time, the top 12 adjective phrases started with either the word "so" or "too". Of those top 12 phrases, 7 of them showed up in the top 12 for both the before 2000 and after 2000 list. For both lists, the first phrase that showed up that didn't start with either "so" or "too" was "not gon", which is presumably an abbreviated form of "not going to". In both lists, 46 of the 50 phrases started with either "so", "too", or "not". Overall though, only 28 of the top 50 adjective phrases in the before 2000 list matched the top 50 adjective phrases from the after list.

In [74]:
# Following our NLTK textbook, chapter on Information Extraction--Chunking (https://www.nltk.org/book/ch07.html)

# Using CHUNKING to parse sentences 
# to look for "adjective phrases", i.e. phrases (or chunks) that have adverbs and adjectives ('RB'+'JJ')
# First step: writing a grammar that defines the POS in the chunk
# we name this grammar "ADJPH" ("ADJective PHrase") using regexes 

#import re #already done in the beginning
grammar_adjph = "ADJPH: {<RB.?>+<JJ.?>}"
# This regex reads as: "find groups ("< >") of RBs (adverbs) together with groups of JJs (adjectives), with groups defineds as
# RBs with any ending (the "." is a placeholder or wildcard for the "R" and the "S" at the end of RBR and RBS, 
# while "?" indicates "optional character" so RB can be found alone as well). Same regex operators apply to JJs.

# Second step: import the nltk parser to process each sentence
chunk_parser_adj = nltk.RegexpParser(grammar_adjph)

#first the before lyrics
B_adjph_tags = []
for sent in BLtagged:
    if len(sent) > 0:
        tree = chunk_parser_adj.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'ADJPH':
                B_adjph_tags.append(subtree)
                
# Visualizing the actual adjective phrase
B_adjective_phrases = []
for sent in B_adjph_tags:
    temp = ''
    for w, t in sent:
        temp += w+ ' '    
    B_adjective_phrases.append(temp)
    
print('First 10 adjective phrases for lyrics before 2000: ', B_adjective_phrases[:10])


# Following our NLTK textbook, chapter 1 on Language Processing (https://www.nltk.org/book/ch01.html)

## FREQUENCY DISTRIBUTIONS
# Top 50 adjective phrases
B_freq_adjph = nltk.FreqDist(B_adjective_phrases)

print('Top adjective phrases by frequency for lyrics before 2000: ')
B_adj_phrase_list=[]
for word, freq in B_freq_adjph.most_common(50):
    #create a list of the most common phrases to use for comparison later
    B_adj_phrase_list.append(word)
    print(word, freq)

            
#print the list of our sentences:
print('Length of adjective phrase sentences for lyrics before 2000: ', len(B_adjph_tags))


First 10 adjective phrases for lyrics before 2000:  ['not nobody ', 'not ready ', 'jelly fish ', 'not ready ', 'too wide ', 'still rock ', 'too sentimental ', 'still rock ', 'still rock ', 'still rock ']
Top adjective phrases by frequency for lyrics before 2000: 
so much  140
so many  120
too legit  102
so good  99
too much  89
so right  58
so bad  55
so fine  52
so hard  50
so long  42
too late  36
so glad  34
too deep  33
so true  32
so strong  31
so deep  31
not gon  28
so sad  27
too many  26
so easy  24
so sweet  23
so wrong  22
so cold  22
little more  22
so high  22
not good  21
not nobody  20
too sexy  20
so real  19
too hot  18
badreally really bad  18
mighty good  18
so much more  17
not last  17
so tight  16
so baby  16
too high  15
not worth  15
so blind  14
so do  14
too low  14
so beautiful  14
so i  14
so confused  13
so excited  13
not tryin  13
so smooth  13
not right  12
so nice  12
very first  12
Length of adjective phrase sentences for lyrics before 2000:  3733


In [75]:
#Now the after lyrics
A_adjph_tags = []
for sent in ALtagged:
    if len(sent) > 0:
        tree = chunk_parser_adj.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'ADJPH':
                A_adjph_tags.append(subtree)
                
# Visualizing the actual adjective phrase
A_adjective_phrases = []
for sent in A_adjph_tags:
    temp = ''
    for w, t in sent:
        temp += w+ ' '    
    A_adjective_phrases.append(temp)
    
print('First 10 adjective phrases for lyrics after 2000: ', A_adjective_phrases[:10])


# Following our NLTK textbook, chapter 1 on Language Processing (https://www.nltk.org/book/ch01.html)

## FREQUENCY DISTRIBUTIONS
# Top 50 adjective phrases
A_freq_adjph = nltk.FreqDist(A_adjective_phrases)

print('Top adjective phrases by frequency for lyrics after 2000: ')
A_adj_phrase_list=[]
for word, freq in A_freq_adjph.most_common(50):
    #create a list of the most common phrases to use for comparison later
    A_adj_phrase_list.append(word)
    print(word, freq)

            
#print the list of our sentences:
print('Length of adjective phrase sentences for lyrics after 2000: ', len(A_adjph_tags))

First 10 adjective phrases for lyrics after 2000:  ['so cool ', 'not good ', 'so smooth ', 'so smooth ', 'so soft ', 'not good ', 'so smooth ', 'so smooth ', 'so smooth ', 'just fine ']
Top adjective phrases by frequency for lyrics after 2000: 
so much  143
too much  118
so good  111
so cold  95
too late  92
so hard  87
so many  71
so bad  68
so high  67
so do  67
so fly  63
too good  46
not gon  43
so baby  38
so crazy  37
just fine  36
not afraid  33
too many  33
so sick  32
not sorry  31
not nobody  30
so right  30
so sweet  29
too long  28
so glad  26
forever young  26
not right  24
so deep  24
so tight  24
feelin so fly  24
so hot  23
not fair  22
so beautiful  22
so wrong  22
so oh  21
so happy  21
too hard  21
not safe  21
so much more  20
so fine  20
so damn  20
so sad  18
too deep  18
not tryna  18
so alive  18
just too little  18
not good  17
so tired  17
so bright  17
too hot  17
Length of adjective phrase sentences for lyrics after 2000:  5720


In [76]:
#####check to see how many phrases are in common for each of the lists
#adjective phrase lists
adj__phrase_common = sum(i in B_adj_phrase_list for i in A_adj_phrase_list)
print(adj__phrase_common)

28


Next, the adverb phrases for before and after the year 2000 lyrics were looked at.For both lists of lyrics, the term "not even" is the most frequent adverb phrases, and for both lists, 24 of the 50 phrases started with either "so"or "not". 6 of the top 10 most frequent adverb phrases in the before 2000 list matched top 10 most frequent adverb phrases in the after list. One phrase in the top 10 most frequent adverb phrases in the before 2000 list, "once again", didn't show up anywhere in the top 50 most freqent adverb phrases in the after 2000 list. Four of the top 10 most frequent adverb phrases in the after 2000 list, "over again", "up all", "so just", and "down down", didn't show up anywhere in the top 50 most frequent adverb phrases in the before 2000 list. In all, however, 32 of the 50 top adverb phrases in each list matched, 4 more matching phrases than the adjective list.

In [78]:
# Now we look for "adverb phrases" or chunks that have 2 consecutive adverbs ('RB')
# First step: writing a grammar that defines POS rules of the adverb phrase the chunk
# we name this grammar "ADVPH" ("ADVerb PHrase")
grammar_advph = "ADVPH: {<RB>+<RB>}"

# Second step: import the nltk parser to process each sentence
chunk_parser_adv = nltk.RegexpParser(grammar_advph)

#first the lyrics before 2000
B_advph_tags = []
for sent in BLtagged:
    if len(sent) > 0:
        tree = chunk_parser_adv.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'ADVPH':
                B_advph_tags.append(subtree)
                
# Visualizing the actual adjective phrase
B_adverb_phrases = []
for sent in B_advph_tags:
    temp = ''
    for w, t in sent:
        temp += w+ ' '    
    B_adverb_phrases.append(temp)
    
print('First 10 adverb phrases for before lyrics: ', B_adverb_phrases[:10])

# top 50 adjective phrases
B_freq_advph = nltk.FreqDist(B_adverb_phrases)

print('Top adverb phrases by frequency for before lyrics: ')
B_adv_phrase_list=[]
for word, freq in B_freq_advph.most_common(50):
    #create a list of the most common phrases to use for comparison later
    B_adv_phrase_list.append(word)
    print(word, freq)

            
#print the list of our sentences:
print('Length of adverb phrase sentences for before lyrics: ', len(B_advph_tags))

First 10 adverb phrases for before lyrics:  ['so far ', 'never enough ', 'once again ', 'never enough ', 'once again ', 'never enough ', 'once again ', 'once again ', 'punk even ', 'too much ']
Top adverb phrases by frequency for before lyrics: 
not even  109
as long  63
not so  55
right now  52
not really  50
not ever  49
so long  41
never ever  41
once again  39
far away  37
not too  29
not no  28
back again  26
too long  25
so much  25
not just  25
really really  24
so far  23
right here  23
as well  23
too far  23
so lonely  21
not alone  20
so now  20
so here  19
ahead now  19
together again  18
badreally really  18
upside down  16
so close  16
so far away  16
so so  16
long ago  15
never really  15
not here  15
so well  14
now here  14
not enough  14
just too  14
as soon  13
back now  13
so baby  13
together forever  13
ever ever  13
so so so  13
not never  12
alone again  12
right there  12
so annie  12
up again  12
Length of adverb phrase sentences for before lyrics:  2781


In [79]:
#now the lyrics after 2000
A_advph_tags = []
for sent in ALtagged:
    if len(sent) > 0:
        tree = chunk_parser_adv.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'ADVPH':
                A_advph_tags.append(subtree)
                
# Visualizing the actual adjective phrase
A_adverb_phrases = []
for sent in A_advph_tags:
    temp = ''
    for w, t in sent:
        temp += w+ ' '    
    A_adverb_phrases.append(temp)
    
print('First 10 adverb phrases for after lyrics: ', A_adverb_phrases[:10])

# top 50 adjective phrases
A_freq_advph = nltk.FreqDist(A_adverb_phrases)

print('Top adverb phrases by frequency for after lyrics: ')
A_adv_phrase_list=[]
for word, freq in A_freq_advph.most_common(50):
    #create a list of the most common phrases to use for comparison later
    A_adv_phrase_list.append(word)
    print(word, freq)

            
#print the list of our sentences:
print('Length of adverb phrase sentences for after lyrics: ', len(A_advph_tags))

First 10 adverb phrases for after lyrics:  ['right now ', 'there just ', 'never enough ', 'now here ', 'back then ', 'not then ', 'then maybe ', 'back then ', 'so easily ', 'not just ']
Top adverb phrases by frequency for after lyrics: 
not even  280
right now  254
not really  143
not ever  143
not never  90
as long  81
over again  65
up all  59
so just  51
down down  46
not so  45
right here  43
not no  43
just too  42
not just  35
so so  34
so close  33
so long  32
so baby  31
not alone  29
as well  28
never ever  28
never really  27
not enough  26
so much  25
so lonely  25
really really  25
feelin so  24
too far  22
just so  22
now now now now now now  22
over now  21
not quite  20
far away  20
right back  20
not always  20
back then  19
not here  19
so far  18
back again  18
down down down  17
only just  17
right there  16
better now better now  16
not there  15
so now  15
not usually  15
da na  14
once again  14
not too  14
Length of adverb phrase sentences for after lyrics:  4397

In [80]:
#####check to see how many phrases are in common for each of the lists
#adjective phrase lists
adv__phrase_common = sum(i in B_adv_phrase_list for i in A_adv_phrase_list)
print(adv__phrase_common)

32


Next, the top 50 single adjectives, adverbs, nouns, and verbs were looked at for both the before 2000 and after 2000 lists. 7 of the top 10 most common adjectives in the before 2000 list were also in the top 10 of the after 2000 list, and the top 2 of each list were the exact same (good, little). Overall, 42 adjectives matched in each list. All 10 of the most common adverbs from the before 2000 list were also the top 10 most common in the after 2000 list, and the top 6 of each list matched exactly (not, so, just, now, never, here). Overall, 43 of the adjectives in each list matched. 8 of the top 10 most common nouns in the before 2000 list matched the list of top 10 nouns in the after 2000 list, with 5 of the nouns landing in the exact same rank of frequency (you, it, me, my, baby). Overall, 44 nouns were in both lists of top 50. 9 of the 10 top most common verbs in the before 2000 list were also in the top 10 of the after list, with only the top word, "is", landing at the same spot in the frequency ranking. 45 of the 50 words in each list matched, however. Not surprisingly, the single list of words for the before and after lyrics had a higher percentage of matches then the list of adjective/adverb phrases that matched. The probably of one word occuring in two different lists of text in general will be more likely than two words appearing consecutively in both lists.

In [82]:
# Top 50 adjective tokens for the before lyrics

B_adjective_tokens = []
for sentence in BLtagged:
    for word, pos in sentence:
        if pos in ['JJ', 'JJR', 'JJS']: # adjective, comparative, superlative
            if len(word)>1:
                B_adjective_tokens.append(word)
B_freq_adjective = nltk.FreqDist(B_adjective_tokens)

B_adj_list=[]
for word, freq in B_freq_adjective.most_common(50):
    #create a list I can later use to compare number of words that are similar in before and after
    B_adj_list.append(word)
    print(word,freq)

good 785
little 766
right 564
more 537
true 482
real 475
much 434
bad 417
wrong 336
new 335
oh 314
hard 289
last 287
better 278
other 273
same 266
big 265
sweet 252
crazy 246
many 235
free 234
best 230
long 222
strong 220
own 220
only 218
old 215
mine 204
high 189
deep 184
whole 183
hot 175
ready 174
sure 174
happy 174
first 163
easy 156
uh 145
ooh 136
fine 136
low 135
wild 135
tight 133
next 130
black 124
alive 121
young 118
cool 106
cold 104
legit 104


In [83]:
# Top 50 adjective tokens for the after lyrics

A_adjective_tokens = []
for sentence in ALtagged:
    for word, pos in sentence:
        if pos in ['JJ', 'JJR', 'JJS']: # adjective, comparative, superlative
            if len(word)>1:
                A_adjective_tokens.append(word)
A_freq_adjective = nltk.FreqDist(A_adjective_tokens)

A_adj_list=[]
for word, freq in A_freq_adjective.most_common(50):
    #create a list I can later use to compare number of words that are similar in before and after
    A_adj_list.append(word)
    print(word,freq)

good 1234
little 852
real 758
bad 753
more 678
right 663
oh 618
better 516
new 512
young 510
hard 509
high 506
low 485
big 469
best 448
same 447
last 436
other 421
hot 417
crazy 403
much 402
wrong 360
old 349
uh 346
ready 317
whole 315
only 302
own 292
next 281
beautiful 265
true 264
long 260
fine 248
cold 248
black 246
ya 244
first 226
nigga 221
free 216
full 211
alive 205
sorry 199
white 198
deep 188
sure 184
many 183
ooh 182
alright 167
late 166
sweet 166


In [84]:
# Top 50 adverb tokens for the before lyrics

B_adverb_tokens = []
for sentence in BLtagged:
    for word, pos in sentence:
        if pos in ['RB', 'RBR', 'RBS']: # adverb, comparative, superlative
            if len(word)>1:
                B_adverb_tokens.append(word)
B_freq_adverb = nltk.FreqDist(B_adverb_tokens)

B_adv_list=[]
for word, freq in B_freq_adverb.most_common(50):
    #create a list I can later use to compare number of words that are similar in before and after
    B_adv_list.append(word)
    print(word,freq)

not 9361
so 3397
just 3078
now 2086
never 1547
here 1008
too 830
then 742
back 740
ever 724
always 722
again 704
away 680
really 640
well 626
there 585
still 574
only 532
long 425
together 387
right 364
even 339
alone 334
more 291
forever 265
down 262
far 261
maybe 247
once 227
else 223
up 211
as 209
enough 177
lonely 176
sometimes 165
close 141
apart 132
better 125
much 120
soon 100
yes 99
very 98
oh 90
anymore 87
around 86
yeah 78
finally 75
somewhere 68
almost 63
everywhere 63


In [85]:
# Top 50 adverb tokens for the after lyrics

A_adverb_tokens = []
for sentence in ALtagged:
    for word, pos in sentence:
        if pos in ['RB', 'RBR', 'RBS']: # adverb, comparative, superlative
            if len(word)>1:
                A_adverb_tokens.append(word)
A_freq_adverb = nltk.FreqDist(A_adverb_tokens)

A_adv_list=[]
for word, freq in A_freq_adverb.most_common(50):
    #create a list I can later use to compare number of words that are similar in before and after
    A_adv_list.append(word)
    print(word,freq)

not 14769
so 4946
just 4778
now 3807
never 2182
here 1426
back 1403
then 1191
too 1116
still 920
ever 895
right 859
away 789
really 785
only 773
again 772
even 767
always 712
up 649
there 566
down 549
well 544
more 528
long 431
alone 399
maybe 372
better 346
else 294
together 290
forever 270
as 248
enough 207
far 205
already 195
sometimes 191
once 185
yeah 172
around 157
lonely 148
pretty 148
probably 140
close 132
na 130
first 128
ya 118
no 114
yes 111
oh 109
apart 108
somewhere 107


In [86]:
# Top 50 noun tokens for the before lyrics

B_noun_tokens = []
for sentence in BLtagged:
    for word, pos in sentence:
        if pos in ['NN', 'NNS', 'NNP', 'NNPS', 'PRP', 'PRP$', 'WP', 'WP$']: # noun, plural, proper, proper plural, personal pronoun, possesive pronoun, WH pronoun, poss WH pronoun
            if len(word)>1:
                B_noun_tokens.append(word)
B_freq_noun = nltk.FreqDist(B_noun_tokens)

B_nn_list=[]
for word, freq in B_freq_noun.most_common(50):
    #create a list I can later use to compare number of words that are similar in before and after
    B_nn_list.append(word)
    print(word,freq)

you 25360
it 10108
me 9872
my 6832
your 5117
we 4341
love 3342
what 2866
baby 2360
time 1927
she 1863
yeah 1757
way 1445
they 1415
night 1346
heart 1247
her 1113
girl 1102
he 1097
who 1092
us 1061
chorus 972
life 965
man 839
day 819
world 710
them 705
eyes 674
our 617
mind 610
something 578
one 562
hey 546
things 536
body 511
nothing 495
thing 475
oh 462
tonight 430
everything 422
him 412
his 390
someone 376
boy 370
ooh 362
ya 339
place 314
home 303
people 301
kind 300


In [87]:
# Top 50 noun tokens for the after lyrics

A_noun_tokens = []
for sentence in ALtagged:
    for word, pos in sentence:
        if pos in ['NN', 'NNS', 'NNP', 'NNPS', 'PRP', 'PRP$', 'WP', 'WP$']: # noun, plural, proper, proper plural, personal pronoun, possesive pronoun, WH pronoun, poss WH pronoun
            if len(word)>1:
                A_noun_tokens.append(word)
A_freq_noun = nltk.FreqDist(A_noun_tokens)

A_nn_list=[]
for word, freq in A_freq_noun.most_common(50):
    #create a list I can later use to compare number of words that are similar in before and after
    A_nn_list.append(word)
    print(word,freq)

you 35090
it 17361
me 15723
my 11845
we 7140
your 6956
what 4473
she 3534
baby 3147
yeah 3005
they 2575
girl 2301
love 2233
time 2070
way 2017
her 1969
them 1956
us 1701
he 1444
life 1383
night 1339
man 1234
who 1107
oh 1066
hey 1061
heart 1038
ya 913
bitch 830
money 822
boy 798
day 796
chorus 770
nothing 757
something 750
thing 746
world 744
body 691
home 672
things 671
ooh 664
our 655
mind 648
everything 647
head 640
eyes 609
tonight 608
shit 595
one 585
name 531
ass 512


In [88]:
# Top 50 verbs tokens for the before lyrics

B_verb_tokens = []
for sentence in BLtagged:
    for word, pos in sentence:
        if pos in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']: # verb, past verb, part gerund verb, verb past part, verb non-3rd, verb 3rd
            if len(word)>1:
                B_verb_tokens.append(word)
B_freq_verb = nltk.FreqDist(B_verb_tokens)

B_vb_list=[]
for word, freq in B_freq_verb.most_common(50):
    #create a list I can later use to compare number of words that are similar in before and after
    B_vb_list.append(word)
    print(word,freq)

is 9589
do 7581
are 5021
be 4338
am 4237
know 3734
want 3515
have 3406
got 2795
let 2028
get 1956
love 1910
go 1835
make 1793
going 1792
say 1680
see 1552
was 1522
feel 1405
come 1288
take 1277
tell 1263
give 1091
need 1054
been 959
keep 770
think 758
did 650
said 635
had 613
hold 597
baby 577
were 534
find 533
hear 472
believe 471
stop 441
gone 412
live 388
put 371
turn 363
show 361
look 358
try 351
leave 341
remember 340
made 335
stay 334
call 319
play 319


In [89]:
# Top 50 verbs tokens for the after lyrics

A_verb_tokens = []
for sentence in ALtagged:
    for word, pos in sentence:
        if pos in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']: # verb, past verb, part gerund verb, verb past part, verb non-3rd, verb 3rd
            if len(word)>1:
                A_verb_tokens.append(word)
A_freq_verb = nltk.FreqDist(A_verb_tokens)

A_vb_list=[]
for word, freq in A_freq_verb.most_common(50):
    #create a list I can later use to compare number of words that are similar in before and after
    A_vb_list.append(word)
    print(word,freq)

is 11748
am 10442
do 9522
are 7875
know 5685
be 5534
got 5466
want 5196
get 4022
have 3429
let 3364
go 3194
going 2516
love 2493
make 2489
see 2447
say 2291
was 2020
take 2012
come 1609
need 1589
feel 1567
been 1481
tell 1481
give 1347
keep 1261
think 1168
said 1035
did 983
had 976
put 933
turn 691
like 680
were 669
gone 666
call 662
stop 657
leave 622
look 621
show 587
try 580
hold 565
made 562
baby 558
find 538
watch 534
hit 514
shake 512
run 492
hear 491


In [90]:
#####check to see how many words are in common for each of the lists
#adjective lists
adj_common = sum(i in B_adj_list for i in A_adj_list)
print(adj_common)

#adverb lists
adv_common = sum(i in B_adv_list for i in A_adv_list)
print(adv_common)

#noun lists
nn_common = sum(i in B_nn_list for i in A_nn_list)
print(nn_common)

#verb lists
vb_common = sum(i in B_vb_list for i in A_vb_list)
print(vb_common)

42
43
44
45


Next, the number of sentences overall that had adjective phrases, adverb phrases, or one of an adjective or adverb phrases, were looked at for the before 2000 and the after 2000 lists. Since there were a different number of total sentences for the before list and after list, the percentage of each list was used for a better comparison. 10% of both the before 2000 and after 2000 lists contained adjective phrases. About 8% of the before 2000 sentences contained an adverb phrase, while the percentage of after 2000 sentences with an adverb phrase was slightly higher at about 9%. About 11% of the before 2000 sentences had either an adjective or adverb phrase, while about 12% of the after 2000 sentences sentences had either an adjective or adverb phrase. Even though the percentage of adverb phrases and adjective or adverb phrases was slightly higher for the after 2000 sentences, overall the percentage for both was pretty close.

In [92]:
# Now we have two lists of POS tags combinations we can compare
# We need to get the sentences back from the tagging exercise and run some stats

#first the before lyrics
# Create a list of original sentences from the ADJECTIVE phrase subset:
B_adjph_whole_sentences = []

# loop over the sentences in the adjective phrase sentences we created:
for sents in B_adjph_tags:
    temp=''
    for (word,tag) in sents:
        temp += word+' '
        B_adjph_whole_sentences.append(temp)
        
print(len(B_adjph_whole_sentences))
#since the total number of sentences is different for before and after lyrics, normalize this number for better comparisons
print(len(B_adjph_whole_sentences)/len(BLtagged))

7850
0.10078186183256088


In [93]:
#now the after lyrics
# Create a list of original sentences from the ADJECTIVE phrase subset:
A_adjph_whole_sentences = []

# loop over the sentences in the adjective phrase sentences we created:
for sents in A_adjph_tags:
    temp=''
    for (word,tag) in sents:
        temp += word+' '
        A_adjph_whole_sentences.append(temp)
        
print(len(A_adjph_whole_sentences))
#normalized
print(len(A_adjph_whole_sentences)/len(ALtagged))

11983
0.10782575832560985


In [94]:
# Create a list of original sentences from the ADVERB phrase subset:
#first the before lyrics
B_advph_whole_sentences = []

# loop over the sentences in the adjective phrase sentences we created:
for sents in B_advph_tags:
    temp=''
    for (word,tag) in sents:
        temp += word+' '
        B_advph_whole_sentences.append(temp)
        
print(len(B_advph_whole_sentences))
#normalized
print(len(B_advph_whole_sentences)/len(BLtagged))

5930
0.07613203065822752


In [95]:
# Create a list of original sentences from the ADVERB phrase subset:
#now the after lyrics
A_advph_whole_sentences = []

# loop over the sentences in the adjective phrase sentences we created:
for sents in A_advph_tags:
    temp=''
    for (word,tag) in sents:
        temp += word+' '
        A_advph_whole_sentences.append(temp)
        
print(len(A_advph_whole_sentences))
#normalized
print(len(A_advph_whole_sentences)/len(ALtagged))

9536
0.0858070960020876


In [96]:
# Combine lists together to have a single list of adjective/adverb phrases:
# Useful to know which sentences are heavy in qualifiers

#first the before lyrics
# create a new variable to store all adjective phrase sentences
#B_adv_adj_phrase_sentences = B_adjph_whole_sentences #this is changing the original list, need to make a copy instead
B_adv_adj_phrase_sentences = B_adjph_whole_sentences.copy()

# iterate over adverb phrase sentences
for sent in B_advph_whole_sentences:
    # if a sentence is not in the adjective phrases list imported
    if sent not in B_adv_adj_phrase_sentences:
        # attach that sentence
        B_adv_adj_phrase_sentences.append(sent)

# print the lenght of the list (i.e. number of sentences with both adjective and adverb phrases) for the before lyrics
print(len(B_adv_adj_phrase_sentences))
#normalized
print(len(B_adv_adj_phrase_sentences)/len(BLtagged))

8735
0.11214389338948016


In [97]:
len(B_adjph_whole_sentences)

7850

In [98]:
#now the after lyrics
# create a new variable to store all adjective phrase sentences
#A_adv_adj_phrase_sentences = A_adjph_whole_sentences #this is changing the original list, need to make a copy instead
A_adv_adj_phrase_sentences = A_adjph_whole_sentences.copy()

# iterate over adverb phrase sentences
for sent in A_advph_whole_sentences:
    # if a sentence is not in the adjective phrases list imported
    if sent not in A_adv_adj_phrase_sentences:
        # attach that sentence
        A_adv_adj_phrase_sentences.append(sent)

# print the lenght of the list (i.e. number of sentences with both adjective and adverb phrases) for the before lyrics
print(len(A_adv_adj_phrase_sentences))
#normalized
print(len(A_adv_adj_phrase_sentences)/len(ALtagged))

12982
0.11681498744747285


In [99]:
len(A_adjph_whole_sentences)

11983

Next, the average length of sentences was compared. The average length of overall sentences before 2000 was about 31 characters, while it was about 33 characters for the after 2000 sentences. The average length of adjective phrases in the before and after lyrics was about 7.3 characters. The average length of adverb phrases for the before sentences was about 8 characters, while adverb phrases in the after list were about 8.1 characters. For both the before 2000 and after 2000 lists, we can see that that the adverb phrases were about 1 character longer.

In [101]:
# Following our NLTK textbook, Writing Structural Programs chapter
# section on Procedural vs Declarative style (http://www.nltk.org/book_1ed/ch04.html) 

## CORPUS STATISTICS--SENTENCES LENGTH

# Calculating the average length of sentences in the before lyrics
# from http://www.nltk.org/book_1ed/ch04.html
total_before = sum(len(sent) for sent in BLSplitR) # remember: 'BLSplitR' is our before lyrics split into sentences
print(total_before / len(BLSplitR))

30.764940750536006


In [102]:
# Calculating the average length of sentences in the after lyrics
# from http://www.nltk.org/book_1ed/ch04.html
total_after = sum(len(sent) for sent in ALSplitR) # remember: 'ALSplitR' is our after lyrics split into sentences
print(total_after / len(ALSplitR))

32.73799861427299


In [103]:
# Calculate the average length of an adjective phrase sentence in the before lyrics
# We can then compare the average length of the adjective phrases to 
# the average sentences we calculated for all sentences in the before lyrics
B_total_adjph_sentences = sum(len(sent) for sent in B_adjph_whole_sentences) # B_adjph_whole_sentences stores our adjective phrases in the before lyrics
print(B_total_adjph_sentences / len(B_adjph_whole_sentences))

7.260636942675159


In [104]:
# Calculate the average length of an adjective phrase sentence in the after lyrics
# We can then compare the average length of the adjective phrases to 
# the average sentences we calculated for all sentences in the after lyrics
A_total_adjph_sentences = sum(len(sent) for sent in A_adjph_whole_sentences) # A_adjph_whole_sentences stores our adjective phrases in the after lyrics
print(A_total_adjph_sentences / len(A_adjph_whole_sentences))

7.3443211215889175


In [105]:
# Calculate the average length of an adverb phrase sentence in the before lyrics
# We can then compare the average length of the adverb phrases to 
# the average sentences we calculated for all sentences in the before lyrics
B_total_advph_sentences = sum(len(sent) for sent in B_advph_whole_sentences) # B_advph_whole_sentences stores our adverb phrases in the before lyrics
print(B_total_advph_sentences / len(B_advph_whole_sentences))

7.990219224283305


In [106]:
# Calculate the average length of an adverb phrase sentence in the after lyrics
# We can then compare the average length of the adjective phrases to 
# the average sentences we calculated for all sentences in the after lyrics
A_total_advph_sentences = sum(len(sent) for sent in A_advph_whole_sentences) # A_advph_whole_sentences stores our adverb phrases in the after lyrics
print(A_total_advph_sentences / len(A_advph_whole_sentences))

8.173972315436242


Finally, sentiment analysis were performed on the sentences overall for the before and after lists, aswell as the adjective phrases, adverb phrases, and adjective/adverb phrases. The sentiment analysis output has 4 values: negative, positive, neutral, and compound. The negative, positive, and neutral values show what percent of words in a sentence are considered either negative, positive, and neutral, and will always add up to 1 (or 100%). The compound value will be between -1 and 1, with scores closer to -1 showing a sentence is more on the negative side, scores closer to 1 showing that a sentence is on the positive side, and a score closer to 0 showing that a sentence is more neutral.

The sentiment analysis for the overall lines of lyrics before 2000 were .06 negative, .13 positive, and .81 neutral, with a compound score of about .08. The overall scores for the after 2000 lyrics were .07 negative, .12 positive, .81 neutral, and a compound score of .05.

For the adjective phrases in the before 2000 lines of lyrics, about .06 were negative, .09 were positive, and .85 were neutral, with a compound score of about .02. The after 2000 adjective phrase lines of lyrics had scores of about .08 negative, .08 positive, and .84 neutral, with a compound score of .008.

For the adverb phrases in the before 2000 lines of lyrics, about .02 were negative, .06 were positive, and .92 were neutral, with a compound score of about .02. The after 2000 adverb phrase lines of lyrics also had scores of about .02 negative, .06 positive, and .92 neutral, with a compound score of .01.

For the adjective/adverb phrases in the before 2000 lines of lyrics, about .06 were negative, .1 were positive, and .84 were neutral, with a compound score of about .02. The after 2000 adjective/adverb phrase lines of lyrics also had scores of about .07 negative, .08 positive, and .85 neutral, with a compound score of .009.

The overall sentences in the before and after 2000 lists had the highest percent of positive words, with the percentage of positive words almost twice as high as the percentage of negative words. The before and after adverb phrases had the highest percentage of neutral words at about .92. The overall before/after 2000 sentences, before/after adjective phrases, and before/after adjective/adverb phrases all had negative scores in the .06-.08 range. Only the before/after adverb phrases had negative percentages that were lower, both of which were about .02. Based on the high neutral score of all the anaylsis, and the close to zero compound scores, all of the lines in the song lyrics appear to balance out to being neutral. At the same time though, they all have higher percentage of positive words then negative words.

In [108]:
#now perform sentiment analysis on before lyrics and after lyrics using the list of before sentences and after sentences
#before sentiment analysis

sid = SentimentIntensityAnalyzer()

BeforeSentAnalysis=pd.DataFrame()

for i in range(0,len(BLSplitR)):
    sent=sid.polarity_scores(BLSplitR[i])
   
    B_Sent=BLSplitR[i]
    Neg=sent['neg']
    Pos=sent['pos']
    Neu=sent['neu']
    Comp=sent['compound']
    TimeForDF=[B_Sent, Neg, Pos, Neu, Comp]
    TempBSent = pd.DataFrame([TimeForDF],columns=('B_Sent','Negative', 'Positive','Neutral','Compound'))
    CombBSent=[BeforeSentAnalysis,TempBSent]
    BeforeSentAnalysis=pd.concat(CombBSent)

In [109]:
BeforeSentAnalysis

B_Sent  Negative  Positive  \
0                          Colour me your colour baby       0.0     0.000   
0                                  Colour me your car       0.0     0.000   
0                       Colour me your colour darling       0.0     0.487   
0                                  I know who you are       0.0     0.000   
0                       Come up off your colour chart       0.0     0.000   
..                                                ...       ...       ...   
0         Your love is like a river peaceful and deep       0.0     0.664   
0   Your soul is like a secret That I could never ...       0.0     0.238   
0   When I look into your eyes I know that it is true       0.0     0.237   
0         God must of spent A little more time On you       0.0     0.208   
0       God must have spent a little more time on you       0.0     0.208   

    Neutral  Compound  
0     1.000    0.0000  
0     1.000    0.0000  
0     0.513    0.5859  
0     1.000    0.0000  
0     1.000    0.0000  
..      ...       ...  
0     0.336    0.8720  
0     0.762    0.3612  
0     0.763    0.4215  
0     0.792    0.2732  
0     0.792    0.2732  

[77891 rows x 5 columns]

In [110]:
AfterSentAnalysis=pd.DataFrame()

for i in range(0,len(ALSplitR)):
    sent=sid.polarity_scores(ALSplitR[i])
   
    A_Sent=ALSplitR[i]
    Neg=sent['neg']
    Pos=sent['pos']
    Neu=sent['neu']
    Comp=sent['compound']
    TimeForDF=[A_Sent, Neg, Pos, Neu, Comp]
    TempASent = pd.DataFrame([TimeForDF],columns=('A_Sent','Negative', 'Positive','Neutral','Compound'))
    CombASent=[AfterSentAnalysis,TempASent]
    AfterSentAnalysis=pd.concat(CombASent)

In [111]:
AfterSentAnalysis

A_Sent  Negative  Positive  \
0            I can feel the magic floating in the air     0.000     0.000   
0                     Being with you gets me that way     0.000     0.000   
0   I watch the sunlight dance across your face an...     0.000     0.000   
0                          Never been this swept away     0.000     0.000   
0       my thoughts just seem to settle on the breeze     0.000     0.000   
..                                                ...       ...       ...   
0                 And I do not want you to see nobody     0.149     0.000   
0              But you are not my boyfriend Boyfriend     0.000     0.000   
0       And I are not your girlfriend Girlfriend yeah     0.000     0.268   
0   But you do not want me to touch nobody else No...     0.118     0.000   
0                  Baby we are not got to tell nobody     0.000     0.000   

    Neutral  Compound  
0     1.000    0.0000  
0     1.000    0.0000  
0     1.000    0.0000  
0     1.000    0.0000  
0     1.000    0.0000  
..      ...       ...  
0     0.851   -0.0572  
0     1.000    0.0000  
0     0.732    0.2960  
0     0.882   -0.0857  
0     1.000    0.0000  

[111133 rows x 5 columns]

In [112]:
BeforeSentAnalysis.describe()

Negative      Positive       Neutral      Compound
count  77891.000000  77891.000000  77891.000000  77891.000000
mean       0.059829      0.127936      0.809359      0.080125
std        0.146551      0.210192      0.243025      0.307270
min        0.000000      0.000000      0.000000     -0.970500
25%        0.000000      0.000000      0.635000      0.000000
50%        0.000000      0.000000      1.000000      0.000000
75%        0.000000      0.247000      1.000000      0.250000
max        1.000000      1.000000      1.000000      0.994500

In [113]:
AfterSentAnalysis.describe()

Negative       Positive        Neutral       Compound
count  111133.000000  111133.000000  111133.000000  111133.000000
mean        0.070564       0.114495       0.812098       0.050499
std         0.157637       0.196414       0.238532       0.312440
min         0.000000       0.000000       0.000000      -0.982800
25%         0.000000       0.000000       0.649000       0.000000
50%         0.000000       0.000000       1.000000       0.000000
75%         0.000000       0.216000       1.000000       0.153100
max         1.000000       1.000000       1.000000       0.994100

In [114]:
#sent analysis on before adjective sentences
BeforeSentAdj=pd.DataFrame()

for i in range(0,len(B_adjph_whole_sentences)):
    sent=sid.polarity_scores(B_adjph_whole_sentences[i])
   
    B_Sent=B_adjph_whole_sentences[i]
    Neg=sent['neg']
    Pos=sent['pos']
    Neu=sent['neu']
    Comp=sent['compound']
    TimeForDF=[B_Sent, Neg, Pos, Neu, Comp]
    TempBSent = pd.DataFrame([TimeForDF],columns=('B_Sent','Negative', 'Positive','Neutral','Compound'))
    CombBSent=[BeforeSentAdj,TempBSent]
    BeforeSentAdj=pd.concat(CombBSent)

In [115]:
#sent analysis on before adverb sentences
BeforeSentAdv=pd.DataFrame()

for i in range(0,len(B_advph_whole_sentences)):
    sent=sid.polarity_scores(B_advph_whole_sentences[i])
   
    B_Sent=B_advph_whole_sentences[i]
    Neg=sent['neg']
    Pos=sent['pos']
    Neu=sent['neu']
    Comp=sent['compound']
    TimeForDF=[B_Sent, Neg, Pos, Neu, Comp]
    TempBSent = pd.DataFrame([TimeForDF],columns=('B_Sent','Negative', 'Positive','Neutral','Compound'))
    CombBSent=[BeforeSentAdv,TempBSent]
    BeforeSentAdv=pd.concat(CombBSent)

In [116]:
#sent analysis on before adjective + adverb sentences
BeforeSentAdjv=pd.DataFrame()

for i in range(0,len(B_adv_adj_phrase_sentences)):
    sent=sid.polarity_scores(B_adv_adj_phrase_sentences[i])
   
    B_Sent=B_adv_adj_phrase_sentences[i]
    Neg=sent['neg']
    Pos=sent['pos']
    Neu=sent['neu']
    Comp=sent['compound']
    TimeForDF=[B_Sent, Neg, Pos, Neu, Comp]
    TempBSent = pd.DataFrame([TimeForDF],columns=('B_Sent','Negative', 'Positive','Neutral','Compound'))
    CombBSent=[BeforeSentAdjv,TempBSent]
    BeforeSentAdjv=pd.concat(CombBSent)

In [117]:
#sent analysis on after adjective sentences
AfterSentAdj=pd.DataFrame()

for i in range(0,len(A_adjph_whole_sentences)):
    sent=sid.polarity_scores(A_adjph_whole_sentences[i])
   
    A_Sent=A_adjph_whole_sentences[i]
    Neg=sent['neg']
    Pos=sent['pos']
    Neu=sent['neu']
    Comp=sent['compound']
    TimeForDF=[A_Sent, Neg, Pos, Neu, Comp]
    TempASent = pd.DataFrame([TimeForDF],columns=('A_Sent','Negative', 'Positive','Neutral','Compound'))
    CombASent=[AfterSentAdj,TempASent]
    AfterSentAdj=pd.concat(CombASent)

In [118]:
#sent analysis on before adverb sentences
AfterSentAdv=pd.DataFrame()

for i in range(0,len(A_advph_whole_sentences)):
    sent=sid.polarity_scores(A_advph_whole_sentences[i])
   
    A_Sent=A_advph_whole_sentences[i]
    Neg=sent['neg']
    Pos=sent['pos']
    Neu=sent['neu']
    Comp=sent['compound']
    TimeForDF=[A_Sent, Neg, Pos, Neu, Comp]
    TempASent = pd.DataFrame([TimeForDF],columns=('A_Sent','Negative', 'Positive','Neutral','Compound'))
    CombASent=[AfterSentAdv,TempASent]
    AfterSentAdv=pd.concat(CombASent)

In [119]:
#sent analysis on before adjective + adverb sentences
AfterSentAdjv=pd.DataFrame()

for i in range(0,len(A_adv_adj_phrase_sentences)):
    sent=sid.polarity_scores(A_adv_adj_phrase_sentences[i])
   
    A_Sent=A_adv_adj_phrase_sentences[i]
    Neg=sent['neg']
    Pos=sent['pos']
    Neu=sent['neu']
    Comp=sent['compound']
    TimeForDF=[A_Sent, Neg, Pos, Neu, Comp]
    TempASent = pd.DataFrame([TimeForDF],columns=('B_Sent','Negative', 'Positive','Neutral','Compound'))
    CombASent=[AfterSentAdjv,TempASent]
    AfterSentAdjv=pd.concat(CombASent)

In [120]:
print(BeforeSentAnalysis.describe())
print(BeforeSentAdj.describe())
print(BeforeSentAdv.describe())
print(BeforeSentAdjv.describe())

           Negative      Positive       Neutral      Compound
count  77891.000000  77891.000000  77891.000000  77891.000000
mean       0.059829      0.127936      0.809359      0.080125
std        0.146551      0.210192      0.243025      0.307270
min        0.000000      0.000000      0.000000     -0.970500
25%        0.000000      0.000000      0.635000      0.000000
50%        0.000000      0.000000      1.000000      0.000000
75%        0.000000      0.247000      1.000000      0.250000
max        1.000000      1.000000      1.000000      0.994500
          Negative     Positive      Neutral     Compound
count  7850.000000  7850.000000  7850.000000  7850.000000
mean      0.062352     0.093028     0.844492     0.022518
std       0.202246     0.248874     0.303439     0.200536
min       0.000000     0.000000     0.000000    -0.700300
25%       0.000000     0.000000     1.000000     0.000000
50%       0.000000     0.000000     1.000000     0.000000
75%       0.000000     0.000000     

In [121]:
print(AfterSentAnalysis.describe())
print(AfterSentAdj.describe())
print(AfterSentAdv.describe())
print(AfterSentAdjv.describe())

            Negative       Positive        Neutral       Compound
count  111133.000000  111133.000000  111133.000000  111133.000000
mean        0.070564       0.114495       0.812098       0.050499
std         0.157637       0.196414       0.238532       0.312440
min         0.000000       0.000000       0.000000      -0.982800
25%         0.000000       0.000000       0.649000       0.000000
50%         0.000000       0.000000       1.000000       0.000000
75%         0.000000       0.216000       1.000000       0.153100
max         1.000000       1.000000       1.000000       0.994100
           Negative      Positive       Neutral      Compound
count  11983.000000  11983.000000  11983.000000  11983.000000
mean       0.075255      0.083357      0.841305      0.008144
std        0.221619      0.240792      0.308482      0.200599
min        0.000000      0.000000      0.000000     -0.811500
25%        0.000000      0.000000      1.000000      0.000000
50%        0.000000      0.000000 